# RGB Channel Alignment — Clean Notebook

Stacked B/G/R plates → aligned RGB.

**Conventions**: `axis=1` = x (columns), `axis=0` = y (rows).

## 1. Imports & Config

In [1]:
import os, glob, csv
import numpy as np
import skimage as sk
import skimage.io as skio

# Paths
INPUT_DIR = './data/images_in'
OUTPUT_DIR = './data/images_out_green'
OFFSETS_CSV = os.path.join(OUTPUT_DIR, 'offsets.csv')

# Defaults
SEARCH_RADIUS = 12
BORDER_RATIO = 0.10
PYR_LAYERS = 5
PYR_SCALE = 2


## 2. Utilities

In [2]:
def center_crop(a: np.ndarray, border: int) -> np.ndarray:
    h, w = a.shape[:2]
    if border <= 0 or 2 * border >= h or 2 * border >= w:
        return a
    return a[border:h-border, border:w-border]

def downsample_block_mean(img: np.ndarray, scale: int) -> np.ndarray:
    img = img.astype(np.float32, copy=False)
    H, W = img.shape
    Hc = (H // scale) * scale
    Wc = (W // scale) * scale
    cropped = img[:Hc, :Wc]
    return cropped.reshape(Hc // scale, scale, Wc // scale, scale).mean(axis=(1, 3))

def sobel_magnitude(img: np.ndarray) -> np.ndarray:
    p = np.pad(img, ((1, 1), (1, 1)), mode="reflect")

    hder = p[:, 2:] - p[:, :-2]
    gx = hder[0:-2, :] + 2.0 * hder[1:-1, :] + hder[2:, :]

    vder = p[2:, :] - p[:-2, :]
    gy = vder[:, 0:-2] + 2.0 * vder[:, 1:-1] + vder[:, 2:]
    
    return np.hypot(gx, gy)


def save_uint8(img: np.ndarray, path: str):
    arr = np.clip(img, 0, 1)
    arr8 = (arr * 255.0).astype(np.uint8)
    skio.imsave(path, arr8)


## 3. Single-Scale Alignment (NCC)

In [3]:
def align_channel(moving: np.ndarray,
                  reference: np.ndarray,
                  start_dx: int = 0,
                  start_dy: int = 0,
                  search_radius: int = 15,
                  border_ratio: float = 0.1,
                  use_gradients: bool = True) -> tuple[np.ndarray, int, int]:

    if use_gradients:
        ref_score_img = sobel_magnitude(reference)
        mov_score_img = sobel_magnitude(moving)
    else:
        ref_score_img = reference
        mov_score_img = moving

    ref_score_img = ref_score_img - ref_score_img.mean()
    mov_score_img = mov_score_img - mov_score_img.mean()

    height, width = ref_score_img.shape
    border_px = int(round(border_ratio * min(height, width)))

    ref_crop = center_crop(ref_score_img, border_px)
    mov_crop = center_crop(mov_score_img, border_px)

    ref_norm = np.linalg.norm(ref_crop) + 1e-12
    mov_norm = np.linalg.norm(mov_crop) + 1e-12
    denom = ref_norm * mov_norm

    best_dx = 0
    best_dy = 0
    best_score = -np.inf

    for dy in range(start_dy - search_radius, start_dy + search_radius + 1):
        shifted_rows = np.roll(mov_crop, dy, axis=0)
        for dx in range(start_dx - search_radius, start_dx + search_radius + 1):
            shifted = np.roll(shifted_rows, dx, axis=1)
            num = np.sum(shifted * ref_crop)
            score = num / denom
            if score > best_score:
                best_score = score
                best_dx = dx
                best_dy = dy

    aligned = np.roll(np.roll(moving, best_dy, axis=0), best_dx, axis=1)
    return aligned, best_dx, best_dy

## 4. Coarse-to-Fine Pyramid Alignment

In [4]:
def align_pyramid_helper(moving: np.ndarray,
                         reference: np.ndarray,
                         search_radius: int = 15,
                         layer: int = 5,
                         scale: int = 2) -> tuple[int, int]:
    if layer == 0:
        _, cur_dx, cur_dy = align_channel(moving, reference, search_radius, border_ratio = 0, use_gradients=False)
        return 2 * cur_dx, 2 * cur_dy
    
    mov_new = downsample_block_mean(moving, scale)
    ref_new = downsample_block_mean(reference, scale)

    start_dx, start_dy = align_pyramid_helper(mov_new, ref_new, search_radius, layer - 1, scale)
    _, cur_dx, cur_dy = align_channel(moving, reference, start_dx, start_dy, search_radius, border_ratio = 0, use_gradients=False)

    return 2 * cur_dx, 2 * cur_dy



def align_pyramid(moving: np.ndarray,
                  reference: np.ndarray,
                  search_radius: int = 15,
                  border_ratio: float = 0.1,
                  layer: int = 5,
                  scale: int = 2,
                  use_gradients: bool = True):
    
    if use_gradients:
        ref_score_img = sobel_magnitude(reference)
        mov_score_img = sobel_magnitude(moving)
    else:
        ref_score_img = reference
        mov_score_img = moving

    ref_score_img = ref_score_img - ref_score_img.mean()
    mov_score_img = mov_score_img - mov_score_img.mean()

    H, W = ref_score_img.shape
    border_px = int(round(border_ratio * min(H, W)))

    ref_crop = center_crop(ref_score_img, border_px)
    mov_crop = center_crop(mov_score_img, border_px)

    best_dx, best_dy = align_pyramid_helper(mov_crop, ref_crop, search_radius, layer, scale)
    best_dx //= 2
    best_dy //= 2
    aligned = np.roll(np.roll(moving, best_dy, axis=0), best_dx, axis=1)
    return aligned, best_dx, best_dy


## 5. Single Image → RGB

In [8]:
def single_image_to_rgb(image_path: str,
                        align_to: str = "green",
                        use_pyramid: bool = False,
                        search_radius: int = 15,
                        border_ratio: float = 0.1,
                        layers: int = 5,
                        scale: int = 2,
                        use_gradients: bool = False):
    img = skio.imread(image_path)
    img = sk.img_as_float(img)
    h = int(np.floor(img.shape[0] / 3.0))
    blue  = img[:h]
    green = img[h:2*h]
    red   = img[2*h:3*h]
    ref = align_to.lower()
    if ref == "green":
        reference = green 
        pairs = [("blue", blue), ("red", red)]
    elif ref == "blue":
        reference = blue
        pairs = [("green", green), ("red", red)]
    else:
        reference = red
        pairs = [("blue", blue), ("green", green)]

    aligned = {"blue": blue, "green": green, "red": red}
    offsets = {"blue": (0,0), "green": (0,0), "red": (0,0)}

    for name, chan in pairs:
        if use_pyramid:
            a, dx, dy = align_pyramid(chan, reference, search_radius, border_ratio, layers, scale, use_gradients)
            method = "pyramid"
        else:
            a, dx, dy = align_channel(chan, reference, 0, 0, search_radius, border_ratio, use_gradients)
            method = "single"
        aligned[name] = a
        offsets[name] = (dx, dy)

    rgb = np.dstack([aligned["red"], aligned["green"], aligned["blue"]])

    return rgb, offsets, method

## 6. Batch Runner (folder → folder + CSV offsets)

In [12]:
def batch_process(input_dir: str,
                  output_dir: str,
                  csv_path: str,
                  align_to: str = "green",
                  search_radius: int = 12,
                  border_ratio: float = 0.10,
                  layers: int = 5,
                  scale: int = 2,
                  use_gradients: bool = False):

    os.makedirs(output_dir, exist_ok=True)

    patterns = ['*.jpg', '*.jpeg', '*.png', '*.tif', '*.tiff']
    files = []
    for pat in patterns:
        files.extend(glob.glob(os.path.join(input_dir, pat)))
    files.sort()

    rows_by_name = {}
    if os.path.exists(csv_path):
        with open(csv_path, 'r', newline='') as f:
            rdr = csv.reader(f)
            header = next(rdr, None)
            for r in rdr:
                if len(r) >= 9:
                    rows_by_name[r[0]] = r

    for src in files:
        ext = os.path.splitext(src)[1].lower()
        use_pyr = ext in ('.tif', '.tiff')

        rgb, offsets, method = single_image_to_rgb(
            image_path=src,
            align_to=align_to,
            use_pyramid=use_pyr,
            search_radius=search_radius,
            border_ratio=border_ratio,
            layers=layers,
            scale=scale,
            use_gradients=use_gradients
        )

        base = os.path.splitext(os.path.basename(src))[0]
        dst = os.path.join(output_dir, f"{base}_aligned.jpg")
        save_uint8(rgb, dst)

        dx_g, dy_g = offsets.get("green", (0, 0))
        dx_r, dy_r = offsets.get("red",   (0, 0))
        dx_b, dy_b = offsets.get("blue",  (0, 0))

        rows_by_name[src] = [src, method,
                             str(dx_g), str(dy_g),
                             str(dx_r), str(dy_r),
                             str(dx_b), str(dy_b),
                             dst]

    with open(csv_path, 'w', newline='') as f:
        w = csv.writer(f)
        w.writerow(['filename','method','dx_g','dy_g','dx_r','dy_r','dx_b','dy_b','out_path'])
        for key in sorted(rows_by_name.keys()):
            w.writerow(rows_by_name[key])

    return list(rows_by_name.values())


## 7. Run All

In [13]:
batch_process(
    input_dir=INPUT_DIR,
    output_dir=OUTPUT_DIR,
    csv_path=OFFSETS_CSV,
    align_to="green",
    search_radius=12,     
    border_ratio=0.10,    
    layers=4,             
    scale=2,
    use_gradients=False               
)



[['./data/images_in/cathedral.jpg',
  'single',
  '0',
  '0',
  '1',
  '7',
  '-2',
  '-5',
  './data/images_out_green/cathedral_aligned.jpg'],
 ['./data/images_in/church.tif',
  'pyramid',
  '0',
  '0',
  '-8',
  '33',
  '-4',
  '-25',
  './data/images_out_green/church_aligned.jpg'],
 ['./data/images_in/emir.tif',
  'pyramid',
  '0',
  '0',
  '17',
  '57',
  '-24',
  '-49',
  './data/images_out_green/emir_aligned.jpg'],
 ['./data/images_in/harvesters.tif',
  'pyramid',
  '0',
  '0',
  '-3',
  '65',
  '-16',
  '-59',
  './data/images_out_green/harvesters_aligned.jpg'],
 ['./data/images_in/icon.tif',
  'pyramid',
  '0',
  '0',
  '5',
  '48',
  '-17',
  '-41',
  './data/images_out_green/icon_aligned.jpg'],
 ['./data/images_in/italil.tif',
  'pyramid',
  '0',
  '0',
  '15',
  '38',
  '-21',
  '-38',
  './data/images_out_green/italil_aligned.jpg'],
 ['./data/images_in/lastochikino.tif',
  'pyramid',
  '0',
  '0',
  '-7',
  '78',
  '2',
  '2',
  './data/images_out_green/lastochikino_aligned

## 8. Single File Operation (Optional)

In [ ]:
IMAGE_PATH = "./data/images_in/emir.tif"
OUT_PATH   = "./images_out/cathedral_aligned.jpg"

In [ ]:
rgb, off, method = single_image_to_rgb(
    image_path=IMAGE_PATH,
    align_to="Green",
    use_pyramid=True,
    search_radius=12,
    border_ratio=0.1,
    layers=4,
    scale=2,
    use_gradients=True
)

In [ ]:
print(f"Blue offset: dx={off['blue'][0]}, dy={off['blue'][1]}")
print(f"Red   offset: dx={off['red'][0]}, dy={off['red'][1]}")
skio.imshow(rgb)
skio.show()

In [ ]:
rgb8 = (np.clip(rgb, 0, 1) * 255).astype(np.uint8)

output_path = 'data/emir_out.jpg'
skio.imsave(output_path, rgb8)